In [1]:
id_insp = 4

In [3]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

In [4]:
acceso = open('../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()

amigocloud = AmigoCloud(token=api_token)
amigocloud

In [5]:
# DATOS DE LAS INPECCION
query = {'query': 'select * from dataset_315614 where id={id_insp}'.format(id_insp=id_insp)}
project = 'https://app.amigocloud.com/api/v1/projects/32082/sql'
select = amigocloud.get(project, query)
data = select['data'][0]

data['fecha_de_registro'] = datetime.strptime(data['fecha_de_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
#data['fecha_quema'] = datetime.strptime(data['fecha_quema'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
#data['fecha_inspeccion'] = datetime.strptime(data['fecha_inspeccion'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')

insp = collections.namedtuple("insp", data.keys())(*data.values())
print('end')

end


In [6]:
insp

insp(fecha_de_registro='15/09/2022', amigo_id='8F78C031E2F9448D92F640BE9ED416E5', ubicacion='0101000020E61000007C9ED8FFED924FC02E417AE1274331C0', cod_prop=19, nom_prop='SANTA_BARBARA', cosechadora=None, codigo_maquina='CJ-3', nombre_operador='PATIN AGUILERA', tiempo_experiencia=5, op_01=True, op_02=True, op_03=True, op_04=True, op_05=True, op_06=True, op_07=True, op_08=True, op_09=True, op_10=True, op_11=True, op_12=True, op_13=True, op_14=False, op_15=True, op_16=True, op_17=True, op_18=True, op_19=True, op_20=True, op_21=True, op_22=False, op_23=True, maq_01=True, maq_02=True, maq_03=True, maq_04=True, maq_05=True, maq_06=True, maq_07=True, maq_08=True, maq_09=True, maq_10=True, maq_11=True, maq_12=True, maq_13=False, canhero='18358 / ROJAS AMELUNGE CARLOS', nota_operador=9.1304347826087, nota_maquina=9.23076923076923, op_01_orientacion='', op_02_orientacion='', op_03_orientacion='', op_04_orientacion='', op_05_orientacion='', op_06_orientacion='', op_07_orientacion='', op_08_orienta

In [7]:
query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_43414 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=insp.amigo_id)}
project = 'https://app.amigocloud.com/api/v1/projects/32082/sql'
fotos = amigocloud.get(project, query_fotos)['data']

#if fotos_filename['data'] != []:
#    dic = {'lote': lote.lote, 'file_name': fotos_filename['data'][0]['s3_filename']}
#    fotos.append(dic)

#fotos

In [8]:
fotos

[{'source_amigo_id': '8F78C031E2F9448D92F640BE9ED416E5',
  's3_filename': 'IMG_0568.jpeg'},
 {'source_amigo_id': '8F78C031E2F9448D92F640BE9ED416E5',
  's3_filename': 'IMG_0566_3xTtgpy.jpeg'},
 {'source_amigo_id': '8F78C031E2F9448D92F640BE9ED416E5',
  's3_filename': 'IMG_0569_8XnkT5p.jpeg'}]

In [9]:
from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests


doc = DocxTemplate("templates/01_template_operador.docx")

#descargar fotos y generar lista InlineImage

lista_fotos_inline = []
for foto in fotos:
    #response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
    response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
    file = open('fotos/' + foto['s3_filename'], "wb")
    file.write(response.content)
    file.close()
    lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor='fotos/' + foto['s3_filename'], width=Mm(50))})

context = {'insp':insp, 'fotos':lista_fotos_inline}

doc.render(context)

# formato de nombre de archivo: "123_CQ_01-01-2022_NOMBRE"
cod_nom = insp.canhero.split(' / ')
file_name = cod_nom[0] + '_EVC_' + insp.fecha_de_registro.replace('/','-') + '_' + cod_nom[1] + '_' + str(insp.id)

#doc.save(r'C:\Users\UTEA\OneDrive - Ingenio Azucarero Guabira S.A (2)\INFORMES\IMFORMES PARA REVISAR\\' + file_name + '.docx')
doc.save(r'_' + file_name + '.docx')